# Fashion MNIST Image Classifier Using the Sequential API

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
tf.__version__

'2.0.0'

In [3]:
keras.__version__

'2.2.4-tf'

In [4]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_full, y_train_full), (x_test, y_test) = fashion_mnist.load_data()

In [5]:
x_train_full.shape

(60000, 28, 28)

In [6]:
x_train_full.dtype

dtype('uint8')

In [7]:
y_train_full.shape

(60000,)

### First lets generate the validation sets
As we are going to use Gradient Descent to train the NN we need to scale the data.</br>
To scale in a range of 0-1 we will divide each element by 255 (maximum value). To make sure we get the result as a float we need to divide by 255.0 instead.

In [8]:
max_value = 255.0
validation_samples = 5000
x_valid, x_train = x_train_full[:validation_samples] / max_value, x_train_full[validation_samples:] / max_value
y_valid, y_train = y_train_full[:validation_samples], y_train_full[validation_samples:]

In [9]:
x_valid.shape

(5000, 28, 28)

In [10]:
x_valid.dtype

dtype('float64')

In [11]:
y_valid.shape

(5000,)

In [12]:
y_valid.dtype

dtype('uint8')

In [13]:
x_train.shape

(55000, 28, 28)

In [14]:
y_train.shape

(55000,)

Each value in Y represents an objet:

In [15]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [16]:
class_names[y_train[0]]

'Coat'

## Building the model
We are creating a Sequential Model. This is the simplest kind of keras model. It consists of a stack of layers sequentially connected, thus its name. </br>

In [17]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300, activation=keras.activations.relu))
model.add(keras.layers.Dense(100, activation=keras.activations.relu))
model.add(keras.layers.Dense(10 , activation=keras.activations.softmax))

### Architecture:
Layer1: Flatten layer. Intput layer that will convert each input into a 1D array (x.reshape(-1,1)).

Layer2: Dense layer. Hidden layer containing all the conecting weights between inputs and the neurons. It uses ReLU as its activation function (hw.b(X) = ACT(XW+B))

Layer3: Dense layer. Hidden layer with 100 neurons with ReLU activation function.

Layer4: Dense later. Output layer with 1 neuron per output (10). It uses the SoftMax activation function because the clases are exclusive. 

In [18]:
# Alternatively we can initialise the model as a list of layers:
model2 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(300, activation=keras.activations.relu),
    keras.layers.Dense(100, activation=keras.activations.relu),
    keras.layers.Dense(10 , activation=keras.activations.softmax)
])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [20]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_4 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


## Compiling the model
We will use sparse_categorical_crossentropy loss because we have a sparse labels (each target class has a unique index). If we had a OneHotMatrix instead, we would use categorical_cossentropy loss. For binary classification we would use sigmoid instead of softmax in the output layer and binary_crossentropy loss.

When using sdg optimizer we are telling Keras to train the model using StocasticGradientDescent when performing the backpropagation algorithm (reverse-mode autodiff). By default sdg optimizer has a learning rate of 0.01. This can be tuned.

#### NOTE:
Use keras.utils.to_categorical() function to convert sparse labels to OneHotVectors.</br>
To go the other way around, use np.argmax() function with axis=1.


In [24]:
model.compile(
    loss=keras.losses.sparse_categorical_crossentropy,
    optimizer=keras.optimizers.SGD(learning_rate=0.01),
    metrics=[keras.metrics.sparse_categorical_accuracy]
             )

## Train the model

In [25]:
history = model.fit(x_train, y_train, epochs=30, validation_data=(x_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/30
55000/55000 [==============================] - 3s 59us/sample - loss: 0.7319 - sparse_categorical_accuracy: 0.7581 - val_loss: 0.5169 - val_sparse_categorical_accuracy: 0.8318
Epoch 2/30
55000/55000 [==============================] - 3s 51us/sample - loss: 0.4882 - sparse_categorical_accuracy: 0.8283 - val_loss: 0.4397 - val_sparse_categorical_accuracy: 0.8550
Epoch 3/30
55000/55000 [==============================] - 3s 51us/sample - loss: 0.4418 - sparse_categorical_accuracy: 0.8449 - val_loss: 0.4162 - val_sparse_categorical_accuracy: 0.8568
Epoch 4/30
55000/55000 [==============================] - 3s 53us/sample - loss: 0.4157 - sparse_categorical_accuracy: 0.8522 - val_loss: 0.4259 - val_sparse_categorical_accuracy: 0.8500
Epoch 5/30
55000/55000 [==============================] - 3s 51us/sample - loss: 0.3956 - sparse_categorical_accuracy: 0.8596 - val_loss: 0.3830 - val_sparse_categorical_accuracy: 0.8644
Epoch 6/30
55000